# train data에 '일반 대화' 추가
- 결과파일명: custom_train.csv

In [2]:
import pandas as pd

base_data_path = '/aiffel/aiffel/workspace/16_dlthon_1/data/'

train = pd.read_csv(base_data_path + 'train.csv')
train.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [2]:
print('전체 갯수',len(train))
print('class 종류:', *set(train['class']))
print('class 협박 대화', len(train[train['class'] == '협박 대화']))
print('class 갈취 대화', len(train[train['class'] == '갈취 대화']))
print('class 직장 내 괴롭힘 대화', len(train[train['class'] == '직장 내 괴롭힘 대화']))
print('class 기타 괴롭힘 대화', len(train[train['class'] == '기타 괴롭힘 대화']))
len(train['conversation'][0])
train['conversation'].apply(len).describe()

전체 갯수 3950
class 종류: 직장 내 괴롭힘 대화 협박 대화 갈취 대화 기타 괴롭힘 대화
class 협박 대화 896
class 갈취 대화 981
class 직장 내 괴롭힘 대화 979
class 기타 괴롭힘 대화 1094


count    3950.000000
mean      226.570886
std       104.560310
min        41.000000
25%       156.000000
50%       203.000000
75%       270.000000
max       874.000000
Name: conversation, dtype: float64

In [3]:
import os

# /aiffel/aiffel/workspace/16_dlthon_1/data/normal_conversation

dirs_normal_text = ['./data/normal_conversation/02_label_data/session2/',
                    './data/normal_conversation/02_label_data/session3/',
                     './data/normal_conversation/02_label_data/session4/']

total_path = []
for dirs in dirs_normal_text:
    for file_path in os.listdir(dirs):
        total_path.append(dirs+file_path)
        
# total path 확인용
# for path in total_path:
#     print(path)

In [4]:
import json

results = []

for path in total_path:
    
    with open(path, 'r') as f:
        json_data = json.load(f)

        for session in json_data['sessionInfo']:

            tmp_text = ''
            for log in session['dialog']:
                tmp_text += log['utterance']

                # 평균 길이보다 크면 break
                if len(tmp_text) >= 226:
                    results.append(tmp_text)
                    tmp_text = ''
                    break
 
print(len(results))
print(results[3])


1100
안녕하세요 5주만에 만나서 반가워요네 반가워요. 요즘 영월에서 전통 민속놀이를 하고 있는데, 보신적 있나요?무슨민속놀이 하는데요 궁금해요단종의 죽음을 기리는 민속놀이라고 합니다.아 슬픈거내요 전 생소해서 그런건슬프긴 하지만 한 번쯤 가서 볼만 합니다. 보러 가세요.민속놀이는 투구던지기 알아요?투구던지기 알죠. 잘 던지시나요?잘던져요 저는 소질이 있나봐요최고 잘 던진 기록은 얼마인가요?백미터정도요 ㅋㅋ 농담이고요 


In [5]:
results_df = pd.DataFrame(
    {
        'idx': range(3950, 3950+len(results)),
        'class': ['일반 대화']*len(results),
        'conversation':results
    }
)

train = pd.concat([train, results_df], ignore_index=True)

In [6]:
# 확인용
print('전체 갯수',len(train))
print('class 종류:', *set(train['class']))
print('class 협박 대화', len(train[train['class'] == '협박 대화']))
print('class 갈취 대화', len(train[train['class'] == '갈취 대화']))
print('class 직장 내 괴롭힘 대화', len(train[train['class'] == '직장 내 괴롭힘 대화']))
print('class 기타 괴롭힘 대화', len(train[train['class'] == '기타 괴롭힘 대화']))
len(train['conversation'][0])
train['conversation'].apply(len).describe()

전체 갯수 5050
class 종류: 기타 괴롭힘 대화 직장 내 괴롭힘 대화 일반 대화 협박 대화 갈취 대화
class 협박 대화 896
class 갈취 대화 981
class 직장 내 괴롭힘 대화 979
class 기타 괴롭힘 대화 1094


count    5050.000000
mean      230.886535
std        93.135962
min        41.000000
25%       169.250000
50%       229.000000
75%       261.000000
max       874.000000
Name: conversation, dtype: float64

In [7]:
train.to_csv('./data/custom_train.csv', index=False)

# test 데이터 csv로
- 결과파일명 : custom_test.csv

In [21]:
import json

with open(base_data_path+'test.json', 'r') as read_file:
    org_datas = json.load(read_file)

test_datas = []

for key in org_datas:
    test_datas.append(org_datas[key]['text'])

test = pd.DataFrame(test_datas,columns=['conversation'])

test.to_csv('./data/custom_test.csv', index=False)

# norm_text 파일 전처리

In [1]:
import os


base_dir = '/aiffel/aiffel/workspace/16_dlthon_1/data/normal_conversation2/test/'

total_path = []

for file_path in os.listdir(base_dir):
    total_path.append(base_dir+file_path)

In [2]:
import json

results = []

for path in total_path:
    with open(path, 'r') as f:
        json_data = json.load(f)

        tmp_text = ''
        for lines in json_data['info'][0]['annotations']['lines']:
            tmp_text += lines['norm_text']
    
            # 평균 길이보다 크면 break
            if len(tmp_text) >= 226:
                results.append(tmp_text)
                tmp_text = ''
                break
            
 
print(len(results))
# print(results[3])


63516


In [3]:
sample = results[4]

print(len(sample))
print(sample)

print(total_path[0])

235
마켓컬리 할인쿠폰은 받았어?아니 어제 들어갔드니 품절이든데이거?어 그런거 같애아쉽네. 나는 어제 받았는데 보니까 적립금도 7000원이나 있더라 하하크크 좋겠다 나는 받을려고 했는데 늦었나봐 안되게근데 적립금없으면 그래도 16000원 써야되는거라서배송비가 붙어서 그리 되는건가 ?응 우리는 신규가 아니라서 4만이상 주문해야 무료배송인데4만이상이라니.그러면 별로 할인율이 안올라가지뭐살지는 골랏어 ? 마켓컬리 대부분 맛있든데
/aiffel/aiffel/workspace/16_dlthon_1/data/normal_conversation2/test/KAKAO_1451_19.json


In [4]:
import pandas as pd

base_data_path = '/aiffel/aiffel/workspace/16_dlthon_1/data/'

train = pd.read_csv(base_data_path + 'train.csv')

import random

samples = random.sample(results, 2000)

len(train)

3950

In [5]:
import re

def preprocess_sentence(sentence, stopwords=None):
    # 개행자 삭제
    sentence = re.sub(r'[\n\r]', ' ', sentence)
    
    # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
    # 예를 들어서 "I am a student." => "I am a student ."와 같이
    # student와 온점 사이에 거리를 만듭니다. 
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # (a-z, A-Z,가-힣,0-9, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
    sentence = re.sub(r"[^a-zA-Z가-힣0-9\.\?\!,]", " ", sentence)
    sentence = sentence.strip()

    # '키키'와 같이 연속된 키를 제거합니다.
    sentence = re.sub(r'키{2,}', '', sentence)
    sentence = re.sub(r'\b키\b', '', sentence)

    if stopwords:
        words = sentence.split()
        filtered_words = [word for word in words if word not in stopwords]
        sentence = ' '.join(filtered_words)

    return sentence

def augmentation(data):
    import pandas as pd
    def split_text(text):
        half_index = len(text) // 2
        return text[:half_index], text[half_index:]
    
    new_data = {
        'idx': [],
        'class': [],
        'conversation': [],
        'label': []
        }

    for idx, row in data.iterrows():
        part1, part2 = split_text(row['conversation'])
        new_data['idx'].append(row['idx'])
        new_data['class'].append(row['class'])
        new_data['conversation'].append(part1)
        new_data['label'].append(row['label'])
        
        new_data['idx'].append(row['idx'])
        new_data['class'].append(row['class'])
        new_data['conversation'].append(part2)
        new_data['label'].append(row['label'])

    new_df = pd.DataFrame(new_data)
    
    return new_df

In [6]:
train['conversation'] = train['conversation'].apply(preprocess_sentence)

In [7]:
# 'class' 값을 기준으로 'label' 컬럼 추가
label_dict = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}

train['label'] = train['class'].map(label_dict)

In [8]:
train = augmentation(train)

In [9]:
results_df = pd.DataFrame(
    {
        'idx': range(3950, 3950+len(samples)),
        'class': ['일반 대화']*len(samples),
        'conversation':samples
    }
)

In [10]:
results_df['conversation'] = results_df['conversation'].apply(preprocess_sentence)

In [11]:
results_df['label'] = results_df['class'].map(label_dict)
results_df.head()

,idx,class,conversation,label
0,3950,일반 대화,집에 오락실에 있는 커다란 게임기 하나 사두고 싶은데 괜찮을까 ? 남자들의 로망이지...,4
1,3951,일반 대화,번화가에 신호가 없어서 교통 질서가 엉망이야 . 나도 매번 갈 때 마다 느껴 . 너...,4
2,3952,일반 대화,수요일에 시간있어 ? 수요일엔 왜 ? 아 언니랑 조카 오거든오랜만에 만나는거 아니니...,4
3,3953,일반 대화,이번에 어떤 병원에서 60대 환자가 죽었대 ! 헐 어쩌다가 그렇게 된 거야 ? 완전...,4
4,3954,일반 대화,나 요즘 지뢰찾기 게임에 빠졌어지뢰찾기 ? 진짜 오랜만에 들어본다 . 지뢰찾기게임 ...,4


In [12]:
train = pd.concat([train, results_df], ignore_index=True)

In [13]:
# 확인용
print('전체 갯수',len(train))
print('class 종류:', *set(train['class']))
print('class 협박 대화', len(train[train['class'] == '협박 대화']))
print('class 갈취 대화', len(train[train['class'] == '갈취 대화']))
print('class 직장 내 괴롭힘 대화', len(train[train['class'] == '직장 내 괴롭힘 대화']))
print('class 기타 괴롭힘 대화', len(train[train['class'] == '기타 괴롭힘 대화']))
print('class 일반 대화', len(train[train['class'] == '일반 대화']))

len(train['conversation'][0])
train['conversation'].apply(len).describe()


전체 갯수 9900
class 종류: 일반 대화 갈취 대화 직장 내 괴롭힘 대화 기타 괴롭힘 대화 협박 대화
class 협박 대화 1792
class 갈취 대화 1962
class 직장 내 괴롭힘 대화 1958
class 기타 괴롭힘 대화 2188
class 일반 대화 2000


count    9900.000000
mean      143.638485
std        70.691633
min        20.000000
25%        88.000000
50%       121.000000
75%       209.000000
max       455.000000
Name: conversation, dtype: float64

In [14]:
train.head()

,idx,class,conversation,label
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거...,0
1,0,협박 대화,. 죄송합니다 . 정말 잘못했습니다 . 너에게는 선택권이 없어 . 선택 못한다면 너...,0
2,1,협박 대화,길동경찰서입니다 . 9시 40분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 ...,0
3,1,협박 대화,. 장난전화는 업무방해죄에 해당됩니다 . 판단은 너에게 달려있다 . 길동경찰서에도...,0
4,2,기타 괴롭힘 대화,너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 ...,3


In [81]:
train.to_csv('./data/custom_train_2.csv', index=False)